# 0.1.0 Load Data
* Codex Data obtained from: http://welikesharingdata.blob.core.windows.net/forshare/index.html
* Manuscript: https://www.cell.com/cell/pdf/S0092-8674(18)30904-8.pdf

In [7]:
import pandas as pd
df = {}

In [2]:
from glob import glob

## Check available Files

In [3]:
glob('../data/big_data/*')

['../data/big_data/channelNames_BALBcMRLdataset.txt',
 '../data/big_data/Additional examples of cell passports - related to FIgure S2.pptx',
 '../data/big_data/CODEX_MRLdataset_neighborhood_graph.csv',
 '../data/big_data/Additional examples of interacting cells - related to Figure S5.pptx',
 '../data/big_data/BALBcMRLdataset_exposuretimes.csv',
 '../data/big_data/zipped_files',
 '../data/big_data/Suppl.Table2.CODEX_paper_MRLdatasetexpression.csv']

# Convert to Parquet

## Expression Data

In [25]:
%%time
df['exp'] = pd.read_csv('../data/big_data/Suppl.Table2.CODEX_paper_MRLdatasetexpression.csv')
new_rows = [str(x) for x in df['exp'].index.tolist()]
df['exp'].index = new_rows
print(df['exp'].shape)

(734101, 38)
CPU times: user 8.76 s, sys: 511 ms, total: 9.27 s
Wall time: 8.56 s


In [ ]:
%%time
df['exp'].transpose().to_parquet('../data/big_data/parquet_files/expression.txt')
df['exp'].to_parquet('../data/big_data/parquet_files/expression-t.txt')

### Load Expression Levels

In [9]:
%%time
df['exp'] = pd.read_csv('../data/big_data/Suppl.Table2.CODEX_paper_MRLdatasetexpression.csv')
print(df['exp'].shape)

(734101, 38)
CPU times: user 9.31 s, sys: 478 ms, total: 9.78 s
Wall time: 9.48 s


In [12]:
df['exp'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734101 entries, 0 to 734100
Data columns (total 38 columns):
Imaging phenotype cluster ID    734101 non-null int64
CD45                            734101 non-null float64
Ly6C                            734101 non-null float64
TCR                             734101 non-null float64
Ly6G                            734101 non-null float64
CD19                            734101 non-null float64
CD169                           734101 non-null float64
CD106                           734101 non-null float64
CD3                             734101 non-null float64
CD1632                          734101 non-null float64
CD8a                            734101 non-null float64
CD90                            734101 non-null float64
F480                            734101 non-null float64
CD11c                           734101 non-null float64
Ter119                          734101 non-null float64
CD11b                           734101 non-null flo

In [13]:
df['exp'].head()

,Imaging phenotype cluster ID,CD45,Ly6C,TCR,Ly6G,CD19,CD169,CD106,CD3,CD1632,...,CD44,NKp46,X.X,Y.Y,Z.Z,MHCII,blank_Cy3_cyc15,blank_Cy5_cyc15,sample_Xtile_Ytile,niche cluster ID
0,9600,1577.675415,-154.301758,130.692184,-4.168493,560.691345,-504.231476,854.670105,-631.294189,385.935242,...,422.408691,515.130066,10,70,13,6712.812988,1665.967896,398.348389,BALBc-3_X05_Y03,32.0
1,9600,1017.838440,-93.069397,144.076584,40.010998,885.595520,-391.357544,62.764454,-474.201172,-469.634583,...,448.701660,171.880310,1000,294,12,2024.678711,1287.959229,421.991425,BALBc-3_X01_Y02,99.0
2,9600,5978.459961,-330.099365,139.631744,-82.840302,1747.897583,-395.508820,954.326782,-1026.204468,3744.718262,...,2229.804443,512.220764,1003,107,8,8647.193359,2817.173828,709.545105,BALBc-2_X05_Y04,74.0
3,9600,6119.109375,-54.384808,-768.871704,25.625927,1065.311890,-485.535431,538.404175,-611.836426,865.842590,...,665.720459,351.108246,1003,113,13,4838.463379,1646.660278,408.523590,BALBc-2_X03_Y04,98.0
4,9600,6272.474609,-235.512405,74.058075,-101.729919,1186.295044,-782.744995,1261.625366,-915.565552,1361.536011,...,2065.742676,259.003235,1003,148,9,5092.891602,2161.109131,713.416199,BALBc-2_X02_Y01,71.0


In [24]:
%%time
df['exp'].transpose().to_parquet('../data/big_data/parquet_files/expression.txt')
df['exp'].to_parquet('../data/big_data/parquet_files/expression-t.txt')

ValueError: parquet must have string column names

In [19]:
%%time
df['tmp'] = pd.read_parquet('../data/processed_data/tmp.parquet')

CPU times: user 1.49 s, sys: 507 ms, total: 2 s
Wall time: 368 ms


In [23]:
df['tmp'].head()

,Imaging phenotype cluster ID,CD45,Ly6C,TCR,Ly6G,CD19,CD169,CD106,CD3,CD1632,...,CD44,NKp46,X.X,Y.Y,Z.Z,MHCII,blank_Cy3_cyc15,blank_Cy5_cyc15,sample_Xtile_Ytile,niche cluster ID
0,9600,1577.675415,-154.301758,130.692184,-4.168493,560.691345,-504.231476,854.670105,-631.294189,385.935242,...,422.408691,515.130066,10,70,13,6712.812988,1665.967896,398.348389,BALBc-3_X05_Y03,32.0
1,9600,1017.838440,-93.069397,144.076584,40.010998,885.595520,-391.357544,62.764454,-474.201172,-469.634583,...,448.701660,171.880310,1000,294,12,2024.678711,1287.959229,421.991425,BALBc-3_X01_Y02,99.0
2,9600,5978.459961,-330.099365,139.631744,-82.840302,1747.897583,-395.508820,954.326782,-1026.204468,3744.718262,...,2229.804443,512.220764,1003,107,8,8647.193359,2817.173828,709.545105,BALBc-2_X05_Y04,74.0
3,9600,6119.109375,-54.384808,-768.871704,25.625927,1065.311890,-485.535431,538.404175,-611.836426,865.842590,...,665.720459,351.108246,1003,113,13,4838.463379,1646.660278,408.523590,BALBc-2_X03_Y04,98.0
4,9600,6272.474609,-235.512405,74.058075,-101.729919,1186.295044,-782.744995,1261.625366,-915.565552,1361.536011,...,2065.742676,259.003235,1003,148,9,5092.891602,2161.109131,713.416199,BALBc-2_X02_Y01,71.0


In [21]:
df['tmp'].shape

(734101, 38)

In [22]:
df['exp'].equals(df['tmp'])

True

In [16]:
df['exp'].columns.tolist()

['Imaging phenotype cluster ID',
 'CD45',
 'Ly6C',
 'TCR',
 'Ly6G',
 'CD19',
 'CD169',
 'CD106',
 'CD3',
 'CD1632',
 'CD8a',
 'CD90',
 'F480',
 'CD11c',
 'Ter119',
 'CD11b',
 'IgD',
 'CD27',
 'CD5',
 'CD79b',
 'CD71',
 'CD31',
 'CD4',
 'IgM',
 'B220',
 'ERTR7',
 'CD35',
 'CD2135',
 'CD44',
 'NKp46',
 'X.X',
 'Y.Y',
 'Z.Z',
 'MHCII',
 'blank_Cy3_cyc15',
 'blank_Cy5_cyc15',
 'sample_Xtile_Ytile',
 'niche cluster ID']

In [15]:
tile_uniq = df['exp']['sample_Xtile_Ytile'].unique()
tile_uniq.shape

(565,)